## Import required libraries

In [22]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
os.chdir('C:\Resume Projects\Traffic_Signs_WebApp-master\Dataset')
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

<>:8: SyntaxWarning: invalid escape sequence '\R'
<>:8: SyntaxWarning: invalid escape sequence '\R'
C:\Users\KIIT\AppData\Local\Temp\ipykernel_20668\3049781081.py:8: SyntaxWarning: invalid escape sequence '\R'
  os.chdir('C:\Resume Projects\Traffic_Signs_WebApp-master\Dataset')


## Load the Model

In [ ]:
# =============================================================================
# Step 1: Install and Import Libraries
# =============================================================================
# pip install numpy pandas matplotlib tensorflow scikit-learn Pillow

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

# =============================================================================
# Step 2: Set the Correct Working Directory
# =============================================================================
try:
    os.chdir(r'C:\Resume Projects\Traffic_Signs_WebApp-master\Dataset')
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print("Error: The specified directory was not found. Please check the path.")


# =============================================================================
# Step 3: Define Constants and Class Names
# =============================================================================
NUM_CLASSES = 43
CLASSES = { 0:'Speed limit (20km/h)', 1:'Speed limit (30km/h)', 2:'Speed limit (50km/h)', 3:'Speed limit (60km/h)', 4:'Speed limit (70km/h)', 5:'Speed limit (80km/h)', 6:'End of speed limit (80km/h)', 7:'Speed limit (100km/h)', 8:'Speed limit (120km/h)', 9:'No passing', 10:'No passing veh over 3.5 tons', 11:'Right-of-way at intersection', 12:'Priority road', 13:'Yield', 14:'Stop', 15:'No vehicles', 16:'Veh > 3.5 tons prohibited', 17:'No entry', 18:'General caution', 19:'Dangerous curve left', 20:'Dangerous curve right', 21:'Double curve', 22:'Bumpy road', 23:'Slippery road', 24:'Road narrows on the right', 25:'Road work', 26:'Traffic signals', 27:'Pedestrians', 28:'Children crossing', 29:'Bicycles crossing', 30:'Beware of ice/snow', 31:'Wild animals crossing', 32:'End speed + passing limits', 33:'Turn right ahead', 34:'Turn left ahead', 35:'Ahead only', 36:'Go straight or right', 37:'Go straight or left', 38:'Keep right', 39:'Keep left', 40:'Roundabout mandatory', 41:'End of no passing', 42:'End no passing veh > 3.5 tons' }


# =============================================================================
# Step 4: Load and Preprocess the Training Data
# =============================================================================
data = []
labels = []
cur_path = os.getcwd()

for i in range(NUM_CLASSES):
    path = os.path.join(cur_path, 'Train', str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(os.path.join(path, a))
            image = image.resize((30, 30))
            data.append(np.array(image))
            labels.append(i)
        except Exception as e:
            print(f"Error loading image {a}: {e}")

data = np.array(data)
labels = np.array(labels)

# --- THE CRITICAL FIX: NORMALIZE THE DATA ---
# This is the essential step that was missing.
data = data.astype('float32') / 255.0

print("Data loading and normalization complete.")
print("Data shape:", data.shape)
print("Labels shape:", labels.shape)


# =============================================================================
# Step 5: Split Data and Prepare for Training
# =============================================================================
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)


# =============================================================================
# Step 6: Build and Compile the CNN Model
# =============================================================================
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


# =============================================================================
# Step 7: Train the Model
# =============================================================================
print("Starting model training with normalized data...")
epochs = 15
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))
print("Model training finished.")


# =============================================================================
# Step 8: Plot Training History
# =============================================================================
# ... (Plotting code is fine, no changes needed) ...
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# =============================================================================
# Step 9: Evaluate and Save the NEW Model
# =============================================================================
from sklearn.metrics import accuracy_score

test_df = pd.read_csv('Test.csv')
test_labels = test_df["ClassId"].values
test_img_paths = test_df["Path"].values

test_data = []
for img_path in test_img_paths:
    full_path = os.path.join(os.getcwd(), img_path)
    image = Image.open(full_path).resize((30,30))
    test_data.append(np.array(image))

X_test_final = np.array(test_data)
# --- NORMALIZE THE FINAL TEST DATA TOO! ---
X_test_final = X_test_final.astype('float32') / 255.0

predictions_probabilities = model.predict(X_test_final)
Y_pred = np.argmax(predictions_probabilities, axis=1)

accuracy = accuracy_score(test_labels, Y_pred)
print(f"Accuracy on the final test dataset: {accuracy * 100:.2f}%")

# Create the save directory if it doesn't exist
# This will be the model/ folder for your Flask app
save_dir = '../model' # Go up one directory, then into 'model'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
model.save(os.path.join(save_dir, "TSR.h5"))
print(f"New model saved to {os.path.join(save_dir, 'TSR.h5')}")

# Step 10 is not needed in the final training script

Successfully changed directory to: C:\Resume Projects\Traffic_Signs_WebApp-master\Dataset
Data loading and normalization complete.
Data shape: (39209, 30, 30, 3)
Labels shape: (39209,)


c:\Resume Projects\Traffic_Signs_WebApp-master\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 26, 26, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 22, 22, 32)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 11, 11, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 11, 11, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 9, 9, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 7, 7, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 43)             │        11,051 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 242,251 (946.29 KB)

 Trainable params: 242,251 (946.29 KB)

 Non-trainable params: 0 (0.00 B)

Starting model training with normalized data...
Epoch 1/15
354/981 ━━━━━━━━━━━━━━━━━━━━ 50s 81ms/step - accuracy: 0.1317 - loss: 3.2955